# 04 – Đánh giá AI Agent & CoT với LangGraph

Xây dựng agent đơn giản với LangGraph, lưu CoT và đánh giá.

In [ ]:
from langgraph.graph import StateGraph
from langchain.chat_models import ChatOpenAI
from deepeval import LLMTestCase, assert_test
from deepeval.metrics.geval import GEval

llm = ChatOpenAI(model_name='gpt-3.5-turbo')

graph = StateGraph(llm=llm)
graph.add_system_message("Bạn là trợ lý toán học, giải thích từng bước.")
graph.add_user_message("Tính 5!?")
result = graph.run()
cot = result.cot
print(cot)

# LogicalFlowMetric
logical_prompt = "Chấm 1 nếu chuỗi suy luận sau hợp lý, 0 nếu không:\n{actual_output}"
LogicalFlowMetric = GEval(name="LogicalFlow", prompt=logical_prompt, threshold=0.7)

# ToolUsageMetric (ví dụ)
tool_prompt = "Chấm 1 nếu agent gọi đúng công cụ, 0 nếu không:\n{actual_output}"
ToolUsageMetric = GEval(name="ToolUsage", prompt=tool_prompt, threshold=0.5)

case = LLMTestCase(input="Tính 5!", actual_output=cot, context=str(result))
assert_test(case, metrics=[LogicalFlowMetric, ToolUsageMetric])